<a href="https://colab.research.google.com/github/meiruv/NIPS-Papers-Analysis/blob/main/NIPS_Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fetching the Data From Kaggle

In [3]:
! pip install -q kaggle

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! cp kaggle.json ~/.kaggle/
! kaggle datasets download -d rowhitswami/nips-papers-1987-2019-updated

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 84% 89.0M/106M [00:01<00:00, 61.3MB/s]
100% 106M/106M [00:01<00:00, 66.8MB/s] 


In [4]:
import os
import zipfile

local_zip = 'nips-papers-1987-2019-updated.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('nips_papers')
zip_ref.close()

In [5]:
!ls nips_papers

authors.csv  papers.csv


In [15]:
import pandas as pd

papers_df = pd.read_csv('nips_papers/papers.csv')
papers_df.head()

,source_id,year,title,abstract,full_text
0,27,1987,Bit-Serial Neural Networks,NaN,573 \n\nBIT - SERIAL NEURAL NETWORKS \n\nAlan...
1,63,1987,Connectivity Versus Entropy,NaN,1 \n\nCONNECTIVITY VERSUS ENTROPY \n\nYaser S...
2,60,1987,The Hopfield Model with Multi-Level Neurons,NaN,278 \n\nTHE HOPFIELD MODEL WITH MUL TI-LEVEL N...
3,59,1987,How Neural Nets Work,NaN,442 \n\nAlan Lapedes \nRobert Farber \n\nThe...
4,69,1987,Spatial Organization of Neural Networks: A Pro...,NaN,740 \n\nSPATIAL ORGANIZATION OF NEURAL NEn...


In [16]:
papers_df.loc[0,'full_text']

'573 \n\nBIT - SERIAL NEURAL  NETWORKS \n\nAlan F.  Murray,  Anthony V . W.  Smith  and Zoe F.  Butler. \n\nDepartment of Electrical Engineering,  University of Edinburgh, \n\nThe King\'s Buildings, Mayfield Road,  Edinburgh, \n\nScotland,  EH93JL. \n\nABSTRACT \n\nA  bit  - serial  VLSI  neural  network  is  described  from  an  initial  architecture  for  a \nsynapse array through to silicon layout and board design.  The issues surrounding bit \n- serial  computation,  and  analog/digital  arithmetic  are  discussed  and  the  parallel \ndevelopment  of  a  hybrid  analog/digital  neural  network  is  outlined.  Learning  and \nrecall  capabilities  are  reported  for  the  bit  - serial  network  along  with  a  projected \nspecification  for  a  64  - neuron,  bit  - serial  board  operating  at 20 MHz.  This tech(cid:173)\nnique  is  extended  to  a  256  (2562  synapses)  network  with  an  update  time  of 3ms, \nusing  a  "paging"  technique  to  time  - multiplex  calculations

# Pre Processing Papers

In [23]:
import spacy
nlp = spacy.load("en",disable=['parser', 'tagger', 'ner'])

In [48]:
def clear_new_line(textt):
  '''
  replaces '\n' with white space and then several white spaces with one.
  '''
  return re.sub(' +', ' ', textt.replace('\n',' '))

In [25]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer("\w+\'?\w+|\w+")

def make_token(textt):
    return tokenizer.tokenize(str(textt))

In [52]:

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words_nltk = stopwords.words('english')

from spacy.lang.en.stop_words import STOP_WORDS

stop_words = set(stop_words_nltk).union(STOP_WORDS)
stop_words.add('https')
stop_words.add('nan')

def remove_stopwords(textt):
    return [token for token in textt if token not in stop_words]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
for i in string.ascii_lowercase:
  if i not in stop_words:
    stop_words.add(i)

In [ ]:
for i in stop_words:
  if len(i)==1:
    print(i)

In [28]:
def lemmatization(textt):
    lemma_result = []
    
    for words in textt:
        doc = nlp(words)
        for token in doc:
            lemma_result.append(token.lemma_)
    return lemma_result

In [43]:
import re
import string
from typing import List

def remove_numbers_from_doc(doc:str)->str:
    """
    Removes numbers from given text.
    """
    if(type(doc)==str):
        text_no_nums = re.sub(r'[0-9]+', '',doc)
        return text_no_nums

In [41]:
papers_df.loc[1,'full_text'].lower()

'1 \n\nconnectivity versus entropy \n\nyaser  s.  abu-mostafa \n\ncalifornia  institute  of technology \n\npasadena, ca 91125 \n\nabstract \n\nhow  does  the  connectivity  of a  neural  network  (number  of synapses  per \nneuron)  relate  to  the complexity  of the  problems  it  can  handle  (measured  by \nthe entropy)?  switching theory would suggest no relation at all, since all boolean \nfunctions  can be  implemented  using  a  circuit  with very  low  connectivity  (e.g., \nusing  two-input  nand  gates).  however,  for  a  network  that  learns  a  problem \nfrom  examples  using  a  local  learning  rule,  we  prove  that  the  entropy  of  the \nproblem becomes  a  lower  bound for  the connectivity of the network. \n\nintroduction \n\nthe most  distinguishing feature of neural networks  is  their  ability to spon(cid:173)\n\ntaneously  learn  the  desired  function  from  \'training\' samples,  i.e.,  their  ability \nto  program themselves.  clearly,  a  given  neural  ne

In [61]:
def pipeline(textt):
  textt1 = remove_numbers_from_doc(textt.lower())
  textt2 = clear_new_line(textt1)
  textt3 = make_token(textt2)
  textt4 = remove_stopwords(textt3)
  return lemmatization(textt4)

In [62]:
%%time
papers_df['abstract'] = papers_df['abstract'].astype(str)

papers_df['abstract_clean'] = papers_df['abstract'].apply(lambda row: pipeline(row))

CPU times: user 28.2 s, sys: 94.1 ms, total: 28.3 s
Wall time: 28.3 s


# LDA

## First Way - Via Gensim

In [33]:
from gensim.corpora import Dictionary

texts = papers_df['abstract_clean']

dictionary_abstracts = Dictionary(texts)
corpus_abstracts = [dictionary_abstracts.doc2bow(text) for text in texts]

import numpy
numpy.random.seed(1) # setting random seed to get the same results each time.

from gensim.models import ldamodel
model_abstracts = ldamodel.LdaModel(corpus_abstracts, id2word=dictionary_abstracts, num_topics=10, minimum_probability=1e-8)
model_abstracts.show_topics()

[(0,
  '0.021*"nan" + 0.016*"capsule" + 0.015*"equivariant" + 0.008*"GCNs" + 0.007*"VQA" + 0.006*"MARL" + 0.006*"BN" + 0.005*"DTW" + 0.004*"Gromov" + 0.004*"GCN"'),
 (1,
  '0.030*"We" + 0.017*"method" + 0.014*"learn" + 0.013*"datum" + 0.012*"network" + 0.010*"train" + 0.009*"base" + 0.008*"time" + 0.007*"propose" + 0.007*"neural"'),
 (2,
  '0.019*"github" + 0.017*"https" + 0.011*"RNN" + 0.010*"Code" + 0.009*"LSTM" + 0.009*"RNNs" + 0.009*"observational" + 0.008*"DNNs" + 0.008*"Transformer" + 0.007*"intervention"'),
 (3,
  '0.034*"algorithm" + 0.015*"problem" + 0.012*"gradient" + 0.010*"function" + 0.008*"In" + 0.008*"matrix" + 0.007*"1" + 0.007*"sample" + 0.007*"convergence" + 0.007*"The"'),
 (4,
  '0.005*"nan" + 0.005*"replay" + 0.004*"VAEs" + 0.004*"SVGD" + 0.004*"ResNet" + 0.004*"Normalization" + 0.003*"GANs" + 0.003*"NTK" + 0.003*"decentralise" + 0.003*"2018"'),
 (5,
  '0.020*"game" + 0.019*"communication" + 0.014*"fairness" + 0.008*"teach" + 0.007*"f" + 0.007*"worker" + 0.007*"stra

In [63]:
from gensim.corpora import Dictionary

texts2 = papers_df['abstract_clean']

dictionary_abstracts2 = Dictionary(texts2)
corpus_abstracts2 = [dictionary_abstracts2.doc2bow(text) for text in texts2]

import numpy
numpy.random.seed(1) # setting random seed to get the same results each time.

from gensim.models import ldamodel
model_abstracts2 = ldamodel.LdaModel(corpus_abstracts2, id2word=dictionary_abstracts2, num_topics=10, minimum_probability=1e-8)
model_abstracts2.show_topics()

[(0,
  '0.026*"algorithm" + 0.022*"problem" + 0.016*"function" + 0.010*"regret" + 0.010*"optimization" + 0.009*"convex" + 0.009*"bound" + 0.009*"set" + 0.009*"result" + 0.009*"optimal"'),
 (1,
  '0.023*"network" + 0.018*"train" + 0.017*"base" + 0.015*"learn" + 0.013*"time" + 0.011*"provide" + 0.011*"image" + 0.010*"neural" + 0.008*"task" + 0.008*"use"'),
 (2,
  '0.032*"agent" + 0.018*"word" + 0.010*"topic" + 0.009*"disentangle" + 0.008*"language" + 0.007*"anchor" + 0.006*"model" + 0.006*"cloud" + 0.006*"plan" + 0.005*"goal"'),
 (3,
  '0.037*"gan" + 0.016*"rnn" + 0.014*"rnns" + 0.014*"transformer" + 0.012*"dnn" + 0.012*"gans" + 0.011*"ode" + 0.010*"discriminator" + 0.010*"sketch" + 0.009*"lstm"'),
 (4,
  '0.020*"block" + 0.013*"path" + 0.011*"observational" + 0.006*"direction" + 0.006*"worker" + 0.005*"admm" + 0.004*"problem" + 0.004*"method" + 0.004*"svgd" + 0.004*"set"'),
 (5,
  '0.036*"method" + 0.024*"datum" + 0.011*"model" + 0.011*"linear" + 0.011*"demonstrate" + 0.008*"propose" + 

## Second Way - Using Scikit Learn

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.9, min_df=2, stop_words='english', token_pattern=r'(?u)\b[A-Za-z]+\b')
papers_tf_idf = tfidf.fit_transform(papers_df['abstract'])

papers_tf_idf

<9680x9946 sparse matrix of type '<class 'numpy.float64'>'
	with 435667 stored elements in Compressed Sparse Row format>

In [66]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=15, random_state=101)

# This can take a while, we are dealing with large number of documents here
LDA.fit(papers_tf_idf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=15, n_jobs=None,
                          perp_tol=0.1, random_state=101, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [67]:
# What topics did we get?

for index, topic in enumerate(LDA.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC #{index}")
    list_keywords = [tfidf.get_feature_names()[index] for index in topic.argsort()[-10:]]
    print(list_keywords)
    print("\n")

THE TOP 15 WORDS FOR TOPIC #0
['clique', 'bernstein', 'abc', 'motifs', 'market', 'schatten', 'mri', 'cloud', 'captions', 'nnz']


THE TOP 15 WORDS FOR TOPIC #1
['vicinity', 'mre', 'byzantine', 'pc', 'couple', 'multiset', 'sbm', 'angular', 'contours', 'contour']


THE TOP 15 WORDS FOR TOPIC #2
['hamiltonian', 'count', 'marginals', 'interventions', 'pomdps', 'mcmc', 'bo', 'copula', 'observational', 'causal']


THE TOP 15 WORDS FOR TOPIC #3
['echo', 'persistence', 'steepest', 'fd', 'phoneme', 'als', 'bcd', 'speaker', 'taxonomy', 'voice']


THE TOP 15 WORDS FOR TOPIC #4
['unitary', 'u', 'nu', 'bn', 'delays', 'kappa', 'mmd', 'cfr', 'vqa', 'lifted']


THE TOP 15 WORDS FOR TOPIC #5
['image', 'deep', 'training', 'network', 'networks', 'neural', 'models', 'data', 'learning', 'model']


THE TOP 15 WORDS FOR TOPIC #6
['violations', 'dcnns', 'volumetric', 'retina', 'assemblies', 'ts', 'attractor', 'buyer', 'mtl', 'captioning']


THE TOP 15 WORDS FOR TOPIC #7
['nnls', 'slam', 'psd', 'quartet', 'com